# Ising spin glass model

## 1D

We study the Hamiltonian of the transverse-field Ising spin glass model without periodic boundary conditions and transverse field `g` set to 0.4.

In [1]:
import Random
using CondensedMatterSOS
@spin σ[1:2]
function hamiltonian(σ)
    Random.seed!(1) # We set the seed so that the results of this example are reproducible.
    return ising_glass_hamiltonian(σ, 0.4, false)
end
hamiltonian(σ)

(-0.4 + 0.0im)σᶻ₂ + (-0.4 + 0.0im)σᶻ₁ + (-0.07058313895389791 - 0.0im)σˣ₁σˣ₂

Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

In [2]:
import Clarabel
solver = Clarabel.Optimizer

Clarabel.MOIwrapper.Optimizer

We can compute a lower bound `-0.8031` to the ground state energy as follow:

In [3]:
function hamiltonian_energy(N, maxdegree, solver; kws...)
    @spin σ[1:N]
    H = hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = NoSparsity())
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 50
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 127
  cones (total) = 2
    : Zero        = 1,  numel = 16
    : PSDTriangle = 1,  numel = 105

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       g

-0.8031077005710117

We can see that the moment matrix uses all monomials:

In [4]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σᶻ₂
 σʸ₂
 σˣ₂
 σᶻ₁
 σʸ₁
 σˣ₁

Using term sparsity with block/cluster completion, we get the same bound:

In [5]:
bound, gram, ν = hamiltonian_energy(2, 2, solver)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 26
  constraints   = 65
  nnz(P)        = 0
  nnz(A)        = 67
  cones (total) = 3
    : Zero        = 1,  numel = 8
    : PSDTriangle = 2,  numel = (21,36)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       

-0.8031077007260203

But with a smaller basis:

In [6]:
[M.basis.monomials for M in ν.blocks]

2-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [1, σᶻ₂, σᶻ₁]
 [σʸ₂, σˣ₂, σʸ₁, σˣ₁]

Using term sparsity with chordal completion, we get a smaller bound:

In [7]:
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = MonomialSparsity(ChordalCompletion()))
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 21
  constraints   = 54
  nnz(P)        = 0
  nnz(A)        = 56
  cones (total) = 6
    : Zero        = 1,  numel = 4
    : PSDTriangle = 5,  numel = (10,10,10,10,10)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dco

-0.8705831380256306

But with an even smaller basis:

In [8]:
[M.basis.monomials for M in ν.blocks]

5-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [σʸ₁, σˣ₁]
 [σˣ₂, σˣ₁]
 [σʸ₂, σˣ₂]
 [1, σᶻ₁]
 [1, σᶻ₂]

## 2D

We now study the same model but on a square lattice.

In [9]:
@spin σ[1:2, 1:2]
hamiltonian(σ)

(-0.4 + 0.0im)σᶻ₂₋₂ + (-0.4 + 0.0im)σᶻ₁₋₂ + (-0.4 + 0.0im)σᶻ₂₋₁ + (-0.4 + 0.0im)σᶻ₁₋₁ + (0.5314767537831963 + 0.0im)σˣ₁₋₂σˣ₂₋₂ + (2.456991333983293 + 0.0im)σˣ₂₋₁σˣ₂₋₂ + (-0.806852326006714 - 0.0im)σˣ₁₋₁σˣ₁₋₂ + (-0.07058313895389791 - 0.0im)σˣ₁₋₁σˣ₂₋₁

We can compute a lower bound `-4.15244` to the ground state energy as follow:

In [10]:
function hamiltonian_energy(N, M, maxdegree, solver; kws...)
    @spin σ[1:N, 1:M]
    H = hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver, sparsity = NoSparsity())
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 170
  constraints   = 418
  nnz(P)        = 0
  nnz(A)        = 430
  cones (total) = 2
    : Zero        = 1,  numel = 67
    : PSDTriangle = 1,  numel = 351

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       

-4.152440484668909

We can see that the moment matrix uses all monomials:

In [11]:
ν.basis.monomials

13-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σᶻ₂₋₂
 σʸ₂₋₂
 σˣ₂₋₂
 σᶻ₁₋₂
 σʸ₁₋₂
 σˣ₁₋₂
 σᶻ₂₋₁
 σʸ₂₋₁
 σˣ₂₋₁
 σᶻ₁₋₁
 σʸ₁₋₁
 σˣ₁₋₁

Using term sparsity with block/cluster completion, we get the same bound:

In [12]:
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver)
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 90
  constraints   = 226
  nnz(P)        = 0
  nnz(A)        = 230
  cones (total) = 3
    : Zero        = 1,  numel = 35
    : PSDTriangle = 2,  numel = (55,136)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost   

-4.152440492472815

But with a smaller basis:

In [13]:
[M.basis.monomials for M in ν.blocks]

2-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [1, σᶻ₂₋₂, σᶻ₁₋₂, σᶻ₂₋₁, σᶻ₁₋₁]
 [σʸ₂₋₂, σˣ₂₋₂, σʸ₁₋₂, σˣ₁₋₂, σʸ₂₋₁, σˣ₂₋₁, σʸ₁₋₁, σˣ₁₋₁]

Using term sparsity with chordal completion, we get a smaller bound:

In [14]:
bound, gram, ν = hamiltonian_energy(2, 2, 2, solver, sparsity = MonomialSparsity(ChordalCompletion()))
bound

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 52
  constraints   = 134
  nnz(P)        = 0
  nnz(A)        = 139
  cones (total) = 10
    : Zero        = 1,  numel = 11
    : PSDTriangle = 9,  numel = (21,21,21,10,...,10)

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost   

-5.465903547590753

But with an even smaller basis:

In [15]:
[M.basis.monomials for M in ν.blocks]

9-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [σˣ₁₋₂, σˣ₂₋₁, σˣ₁₋₁]
 [1, σᶻ₂₋₁, σᶻ₁₋₁]
 [σˣ₂₋₂, σˣ₁₋₂, σˣ₂₋₁]
 [σʸ₁₋₁, σˣ₁₋₁]
 [σʸ₂₋₁, σˣ₂₋₁]
 [σʸ₁₋₂, σˣ₁₋₂]
 [1, σᶻ₁₋₂]
 [σʸ₂₋₂, σˣ₂₋₂]
 [1, σᶻ₂₋₂]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*